In [1]:
!pip install emot
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import string
import re
import itertools
import emot
from math import sqrt
import tensorflow as tf
from tensorflow.keras import layers, Sequential, Model, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking, Bidirectional, Flatten, SpatialDropout1D, Dropout, GRU, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
import transformers
from tensorflow.keras.utils import plot_model
from tqdm.auto import tqdm
from gensim.utils import tokenize as gensim_tokenize
from gensim import models
from collections import defaultdict
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import contractions
import math

# Configuration
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 200
tqdm.pandas()
contractions.add('e.g.', 'for example')
contractions.add('e.g.,', 'for example')
contractions.add('T.G.I.F.', 'thank God it is Friday')

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
print("Tensorflow version " + tf.__version__)
from kaggle_datasets import KaggleDatasets

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU detected")
except ValueError:
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    print("GPU detected")

print("Number of accelerators: ", strategy.num_replicas_in_sync)
GCS_PATH = KaggleDatasets().get_gcs_path("mbti-type") # you can list the bucket with "!gsutil ls $GCS_PATH"
print(GCS_PATH)

Tensorflow version 2.16.1
GPU detected
Number of accelerators:  1
gs://kds-6d755019c817a6dbab29acfe4022ffa8206740ed3406b45711905b10


In [4]:
emot.EMOTICONS_EMO

{':‑)': 'Happy face or smiley',
 ':-))': 'Very happy',
 ':-)))': 'Very very Happy face or smiley',
 ':)': 'Happy face or smiley',
 ':))': 'Very Happy face or smiley',
 ':)))': 'Very very Happy face or smiley',
 ':-]': 'Happy face or smiley',
 ':]': 'Happy face or smiley',
 ':-3': 'Happy face smiley',
 ':3': 'Happy face smiley',
 ':->': 'Happy face smiley',
 ':>': 'Happy face smiley',
 '8-)': 'Happy face smiley',
 ':o)': 'Happy face smiley',
 ':-}': 'Happy face smiley',
 ':}': 'Happy face smiley',
 ':-)': 'Happy face smiley',
 ':c)': 'Happy face smiley',
 ':^)': 'Happy face smiley',
 '=]': 'Happy face smiley',
 '=)': 'Happy face smiley',
 ':‑D': 'Laughing, big grin or laugh with glasses',
 ':D': 'Laughing, big grin or laugh with glasses',
 '8‑D': 'Laughing, big grin or laugh with glasses',
 '8D': 'Laughing, big grin or laugh with glasses',
 'X‑D': 'Laughing, big grin or laugh with glasses',
 'XD': 'Laughing, big grin or laugh with glasses',
 '=D': 'Laughing, big grin or laugh with glass

In [5]:
emot.UNICODE_EMOJI

{'🥇': ':1st_place_medal:',
 '🥈': ':2nd_place_medal:',
 '🥉': ':3rd_place_medal:',
 '🆎': ':AB_button_(blood_type):',
 '🏧': ':ATM_sign:',
 '🅰': ':A_button_(blood_type):',
 '🇦🇫': ':Afghanistan:',
 '🇦🇱': ':Albania:',
 '🇩🇿': ':Algeria:',
 '🇦🇸': ':American_Samoa:',
 '🇦🇩': ':Andorra:',
 '🇦🇴': ':Angola:',
 '🇦🇮': ':Anguilla:',
 '🇦🇶': ':Antarctica:',
 '🇦🇬': ':Antigua_&_Barbuda:',
 '♒': ':Aquarius:',
 '🇦🇷': ':Argentina:',
 '♈': ':Aries:',
 '🇦🇲': ':Armenia:',
 '🇦🇼': ':Aruba:',
 '🇦🇨': ':Ascension_Island:',
 '🇦🇺': ':Australia:',
 '🇦🇹': ':Austria:',
 '🇦🇿': ':Azerbaijan:',
 '🔙': ':BACK_arrow:',
 '🅱': ':B_button_(blood_type):',
 '🇧🇸': ':Bahamas:',
 '🇧🇭': ':Bahrain:',
 '🇧🇩': ':Bangladesh:',
 '🇧🇧': ':Barbados:',
 '🇧🇾': ':Belarus:',
 '🇧🇪': ':Belgium:',
 '🇧🇿': ':Belize:',
 '🇧🇯': ':Benin:',
 '🇧🇲': ':Bermuda:',
 '🇧🇹': ':Bhutan:',
 '🇧🇴': ':Bolivia:',
 '🇧🇦': ':Bosnia_&_Herzegovina:',
 '🇧🇼': ':Botswana:',
 '🇧🇻': ':Bouvet_Island:',
 '🇧🇷': ':Brazil:',
 '🇮🇴': ':British_Indian_Ocean_Territory:',
 '🇻🇬': ':British_Vir

In [6]:
UNICODE_EMO = {v: k[1:-1] for v, k in emot.UNICODE_EMOJI.items()}


### 1. `convert_emojis(text)`
**Purpose**: This function replaces emojis in the text with their descriptive meanings using the `UNICODE_EMO` dictionary.

### 2. `convert_emoticons(text)`
**Purpose**: Replaces emoticons (e.g., `:)`, `:(`) in the text with their descriptive meanings. It uses regular expressions to accurately match and replace emoticons, even with special characters.

### 3. `process_and_tokenize(text)`
**Purpose**: A comprehensive preprocessing function that:
- Replaces URLs with the token `'url'`
- Converts emojis and emoticons into descriptive words
- Expands contractions (like "don't" to "do not")
- Tokenizes text into individual words

### 4. `get_mbti_vector(mbti_type)`
**Purpose**: Converts an MBTI personality type (e.g., `INTJ`) into a binary vector, with 1 representing the presence of a trait and 0 representing its opposite.

### 5. `plot_num_posts_by_indicator(df)`
**Purpose**: Visualizes the number of posts by each MBTI indicator (e.g., Introversion vs. Extroversion). It creates subplots for each trait pair to help analyze dataset balance.

### 6. `plot_num_posts_by_length(title, df)`
**Purpose**: Plots the distribution of posts by sentence length (number of tokens). Helps in understanding the dataset's text length characteristics and any outliers.

### 7. `plot_num_posts_by_type(df)`
**Purpose**: Visualizes the number of posts for each MBTI type. Useful for assessing the type distribution in the dataset, which can highlight class imbalance.

### 8. `get_length_filtered_df(df, token_len, min_tokens)`
**Purpose**: Filters the dataset to include only posts with a specified minimum number of tokens, helping to refine the data by removing very short posts.

### 9. `get_token_freq(df)`
**Purpose**: Calculates the frequency of each token (word) across all posts. Returns a `token_freq` dictionary and a series (`token_freq_ser`) that groups tokens by frequency, which is useful for token analysis.

### 10. `get_token_analysis(min_freq)`
**Purpose**: Analyzes token frequency based on a minimum frequency threshold, calculating the percentage of words and tokens that meet or exceed this threshold. 

### 11. `plot_changes()`
**Purpose**: Plots changes in word retention based on different minimum frequency thresholds. It provides insights into token distribution and the impact of frequency filtering.



In [7]:
def convert_emojis(text):
    for emoji, meaning in UNICODE_EMO.items():
        text = text.replace(emoji, meaning)
    return text

def convert_emoticons(text):
    for emoticon, meaning in emot.EMOTICONS_EMO.items():
        # Escape special characters in emoticons to use them literally in regex
        emoticon = re.escape(emoticon)
        text = re.sub(emoticon, meaning, text)
        text = re.sub(emoticon.lower(), meaning, text)
    return text

def process_and_tokenize(text):
    """
    1. Replace URLs in text
    2. Replace emojis and emoticons with their meanings
    3. Replace contractions with expanded forms
    4. Tokenize text
    """
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'url', text).strip()
    text = convert_emojis(text)
    text = convert_emoticons(text)
    text = contractions.fix(text)
    return list(gensim_tokenize(text, lower=True))

def get_mbti_vector(mbti_type):
    I, N, F, P = mbti_type
    mbti_vector = [I == 'I', N == 'N', F == 'F', P == 'P']
    return np.array(mbti_vector).astype(int)

def plot_num_posts_by_indicator(df):
    fig = make_subplots(rows=2, cols=2)
    for i, (one, opp) in enumerate(zip(one_slice, opposite)):
        counts = df.groupby(one).type.count().sort_index()
        row = 1 if i <= 1 else 2
        col = 1 if i in [0, 2] else 2
        fig.append_trace(go.Bar(x=[opp, one], y=counts.tolist()), row=row, col=col)
    fig.update_layout(showlegend=False, title='Number of posts split by each indicator')
    fig.show()

def plot_num_posts_by_length(title, df):
    token_len = df.tokens.apply(len)
    token_len_dist = df.groupby(token_len).type.count()
    fig = px.bar(token_len_dist.reset_index(), x='tokens', y='type', title=title)
    fig.update_layout(xaxis_title="Sentence Length (Num. of tokens)", yaxis_title="Count")
    fig.show()
    return token_len, token_len_dist

def plot_num_posts_by_type(df):
    type_posts = df.groupby('type').posts.count().sort_values(ascending=False).reset_index()
    fig = px.bar(type_posts, x='type', y='posts', title='Number of posts by MBTI type')
    fig.update_layout(xaxis_title='MBTI Type', yaxis_title='Count')
    fig.show()

def get_length_filtered_df(df, token_len, min_tokens):
    return df[token_len >= min_tokens]

def get_token_freq(df):
    token_freq = defaultdict(int)
    for token_list in df.tokens:
        for token in token_list:
            token_freq[token] += 1
    token_freq = pd.Series(token_freq)
    token_freq_ser = token_freq.groupby(token_freq).count()
    token_freq_ser.name = 'NumTokens'
    token_freq_ser.index.name = 'Freq'
    return token_freq, token_freq_ser

def get_token_analysis(min_freq):
    total_words = (token_freq_ser.index * token_freq_ser).sum()
    total_num_tokens = token_freq_ser.sum()
    invalid_token_freq = token_freq_ser[token_freq_ser.index <= min_freq]
    invalid_total_words = (invalid_token_freq.index * invalid_token_freq).sum()
    invalid_tokens = invalid_token_freq.sum()
    return round(invalid_total_words/total_words*100, 2), round(invalid_tokens/total_num_tokens*100, 2), total_num_tokens - invalid_tokens

def plot_changes():
    min_freqs = list(range(0, 11))
    total_tokens_list, unique_tokens_list, valid_tokens_list = [], [], []
    for min_freq in min_freqs:
        total_tokens, unique_tokens, valid_tokens = get_token_analysis(min_freq)
        total_tokens_list.append(100 - total_tokens)
        unique_tokens_list.append(100 - unique_tokens)
        valid_tokens_list.append(valid_tokens)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=min_freqs, y=total_tokens_list , name="Percentage of total number of words remaining"))
    fig.add_trace(go.Scatter(x=min_freqs, y=unique_tokens_list, name="Percentage of number of unique tokens remaining"))
    fig.update_layout(showlegend=True, legend=dict(yanchor="top", y=0.75, xanchor="right", x=0.99), xaxis_title="Min. Freq. Count",
                      yaxis_title="Percentage")
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=min_freqs, y=valid_tokens_list , name="Valid unique tokens"))
    fig.update_layout(showlegend=True, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99), xaxis_title="Min. Freq. Count",
                      yaxis_title="Count of Valid Unique Tokens")
    fig.show()


1. **Define MBTI Indicators**: `one_slice` contains one side of each MBTI indicator (e.g., `I` for Introversion), and `opposite` contains the corresponding opposites (`E` for Extroversion).

2. **Initialize Data Length List**: `df_lens` will store the length of the dataset, useful for tracking changes during preprocessing.

3. **Load Dataset**: Reads the MBTI dataset into `dfposts`.

4. **Process Posts**:
   - Strips brackets (`[]`) around each "posts" entry.
   - Splits posts by the delimiter "|||", creating a list of individual posts for each user.

5. **Expand Posts**: `explode` is used to turn each post in the list into its own row, making the data easier to work with.

6. **Tokenize Text**: Applies `process_and_tokenize` to each post to convert text into tokens.

7. **Add MBTI Trait Columns**: `get_mbti_vector` creates binary columns for each trait (e.g., `I` vs. `E`) based on the MBTI type, aiding in analysis and model training.

In [8]:
one_slice = ['I', 'N', 'F', 'P']  # One slice of the indicators
opposite = ['E', 'S', 'T', 'J']  # Other end of the respective indicators
df_lens = []

In [9]:
dfposts = pd.read_csv('/kaggle/input/mbti-type/mbti_1.csv')
df_lens.append(len(dfposts))
dfposts.loc[:, 'posts'] = dfposts['posts'].apply(lambda x: x[1:-1].split('|||'))
dfposts = dfposts.explode('posts', ignore_index=True)
dfposts['tokens'] = dfposts.posts.progress_apply(process_and_tokenize)
dfposts[one_slice] = dfposts.type.apply(get_mbti_vector).tolist()

  0%|          | 0/422845 [00:00<?, ?it/s]

### Explanation and Teaching

This code cell provides a detailed preprocessing pipeline for the MBTI dataset, refining and visualizing the text data. Here’s a breakdown:

1. **Token Frequency Calculation**: 
   - `get_token_freq(dfposts)` calculates the frequency of each token (word) in the dataset.
   - `plot_changes()` then plots how token count changes when applying minimum frequency thresholds. This analysis helps identify rare words that may add noise rather than value.

2. **Token Filtering by Frequency**:
   - `min_freq = 8` sets a minimum frequency threshold, filtering out tokens that appear fewer than eight times. 
   - This improves model focus by retaining only commonly occurring words, reducing data noise and computational complexity.

3. **Sentence Length Distribution Analysis**:
   - `plot_num_posts_by_length` visualizes the distribution of sentence lengths before and after filtering, which helps in understanding the text length characteristics and in identifying excessively short posts.
   - A minimum sentence length (`min_tokens = 5`) is applied, filtering out very short sentences that might lack meaningful content.

4. **Data Reduction with Filtering**:
   - `prev_len` and `df_lens` store the count of posts before and after filtering, and the count of removed posts. This data is useful for tracking how preprocessing impacts dataset size.
   - A Sankey diagram is then created to visually display changes in dataset size throughout each preprocessing stage. This aids in understanding the dataset reduction flow and validating that meaningful data has been retained.

5. **Visualizing MBTI Type and Indicator Distributions**:
   - `plot_num_posts_by_type(dfposts)` and `plot_num_posts_by_indicator(dfposts)` plot the distribution of posts by MBTI type and by each personality indicator, respectively.
   - These visualizations reveal any class imbalances, which can guide model training strategies (e.g., class weighting or data augmentation).

### Teaching Points

- **Token Frequency Filtering**: Removing rare tokens can make the model more efficient and less prone to overfitting on infrequent words.
- **Length-based Filtering**: Filtering out very short posts helps focus the model on more substantial text content, improving training quality.
- **Tracking Dataset Reduction**: Using a Sankey diagram to visualize data flow through preprocessing is helpful for understanding how each step impacts data volume and for ensuring data integrity.
- **Distribution Analysis**: Examining distributions by MBTI type and indicator highlights imbalances that may need to be addressed for balanced model training.

This code cell, therefore, covers essential preprocessing and data analysis tasks, helping ensure that the dataset is optimized for robust MBTI type prediction.

In [10]:
token_freq, token_freq_ser = get_token_freq(dfposts)
plot_changes()

In [11]:
min_freq = 8
vocab = set(token_freq[token_freq > min_freq].index)
dfposts['tokens'] = dfposts.tokens.apply(lambda x: list(filter(lambda y: y in vocab, x)))

In [12]:
token_len, token_len_dist = plot_num_posts_by_length('Sentence length distribution before processing', dfposts)

In [13]:
min_tokens = 5  # min sentence length
_ = plot_num_posts_by_length('Sentence length distribution after processing', get_length_filtered_df(dfposts, token_len, min_tokens))

In [14]:
prev_len = len(dfposts)
dfposts = get_length_filtered_df(dfposts, token_len, min_tokens)
df_lens.extend([len(dfposts), prev_len-len(dfposts)])

In [15]:
nodes = ['Initial', 'Explosion', 'Valid Length', 'Invalid Length']
colors = ['rgba(0, 127, 0, 0.8)', 'rgba(0, 255, 0, 0.8)', 'rgba(0, 0, 191, 0.8)', 'rgba(191, 0, 0, 0.8)']

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label = nodes,
      color = colors
    ),
    link = dict(
      source = [0, 1, 1],
      target = [1, 2, 3],
      value = df_lens,
))])


fig.update_layout(title_text='How the number of data points change over the process')
fig.show()

In [16]:
plot_num_posts_by_type(dfposts)
plot_num_posts_by_indicator(dfposts)

In [17]:
max_tokens = 40  # max sentence length
target = dfposts[one_slice].to_numpy()

### Explanation and Teaching

This code cell focuses on preparing the text data for model training by creating a vocabulary, indexing tokens, padding sequences, and splitting the data into training and testing sets. Here's a breakdown:

1. **Vocabulary Creation**:
   - `vocab = set(w for l in dfposts.tokens for w in l)` constructs a set of unique tokens from the tokenized posts. Using a set ensures that each word is only represented once, resulting in a unique vocabulary.
   - `vocab = {w: i + 1 for i, w in enumerate(vocab)}` creates a dictionary that maps each unique token to an index. The `enumerate` function is used to assign each word a unique integer ID, starting from 1. Indexing words is crucial for converting text data into numerical format, which is necessary for machine learning models.

2. **Token Indexing**:
   - `dfposts.loc[:, 'token_idxs'] = dfposts.tokens.apply(lambda l: [vocab[w] for w in l])` replaces each token in the tokenized posts with its corresponding index from the `vocab` dictionary. This transformation enables the model to process text as numerical input.

3. **Sequence Padding**:
   - `features = pad_sequences(dfposts['token_idxs'], maxlen=max_tokens, value=0., padding='pre', truncating='pre')` pads the token index sequences to ensure uniform input length across all samples. 
   - The `maxlen` parameter specifies the desired length for each sequence; sequences shorter than this length are padded with zeros (or a specified value), while longer sequences are truncated from the beginning if they exceed this length. Padding is essential for batch processing in neural networks.

4. **Data Splitting**:
   - `x_train, x_test, y_train, y_test = train_test_split(features, target, train_size=0.9, shuffle=True)` splits the padded features and target labels into training and testing sets. 
   - The `train_size=0.9` argument indicates that 90% of the data will be used for training, while 10% will be reserved for testing. The `shuffle=True` argument ensures that the data is randomly shuffled before splitting, which is important for avoiding any order biases in the dataset.

### Teaching Points

- **Creating a Vocabulary**: Constructing a unique vocabulary and indexing tokens is a fundamental step in natural language processing (NLP) as it transforms text into a numerical format suitable for machine learning models.
- **Token Indexing**: Mapping tokens to indices allows models to interpret text data numerically, which is a prerequisite for feeding data into most machine learning algorithms.
- **Padding Sequences**: Padding ensures that all input sequences have the same length, which is critical for batch processing in deep learning models. This step is often required when working with recurrent neural networks (RNNs) or convolutional neural networks (CNNs) for text.
- **Data Splitting**: Dividing the dataset into training and testing sets allows for evaluating the model's performance on unseen data, which helps in assessing its generalization capabilities.

Overall, this cell efficiently prepares the dataset for training a machine learning model by ensuring that the text data is properly formatted, indexed, and split into appropriate training and testing subsets.

In [18]:
vocab = set(w for l in dfposts.tokens for w in l)


In [19]:
vocab

{'slides',
 'wage',
 'telepathic',
 'fanciful',
 'cyberpunk',
 'sharing',
 'douchebags',
 'maddening',
 'bittersweet',
 'percieving',
 'socionics',
 'inputs',
 'stir',
 'adjustments',
 'hopeful',
 'examining',
 'tmi',
 'alternating',
 'windows',
 'lightbulb',
 'prioritize',
 'opinion',
 'yay',
 'individual',
 'sela',
 'fakeness',
 'ww',
 'depression',
 'sw',
 'hayley',
 'professionals',
 'verify',
 'possession',
 'volunteers',
 'pauses',
 'swiss',
 'retard',
 'alive',
 'unfiltered',
 'seat',
 'dum',
 'atheistic',
 'study',
 'alarmed',
 'injustices',
 'meant',
 'combinations',
 'pls',
 'recognised',
 'printed',
 'manageable',
 'prayer',
 'tilt',
 'curiosity',
 'coelho',
 'disciplined',
 'speculations',
 'handbsurprisedk',
 'hesitantkellie',
 'continuously',
 'tesla',
 'disintegrate',
 'lords',
 'introduces',
 'fm',
 'teams',
 'goaty',
 'admin',
 'rang',
 'duh',
 'beginning',
 'bats',
 'novelty',
 'pressure',
 'vice',
 'pounce',
 'batting',
 'responsibly',
 'welsh',
 'downright',
 'raspb

In [20]:
vocab = {w: i + 1 for i, w in enumerate(vocab)}
dfposts.loc[:, 'token_idxs'] = dfposts.tokens.apply(lambda l: [vocab[w] for w in l])
features = pad_sequences(dfposts['token_idxs'], maxlen=max_tokens, value=0., padding='pre', truncating='pre')
x_train, x_test, y_train, y_test = train_test_split(features, target, train_size=0.9, shuffle=True)

In [21]:
vocab

{'slides': 1,
 'wage': 2,
 'telepathic': 3,
 'fanciful': 4,
 'cyberpunk': 5,
 'sharing': 6,
 'douchebags': 7,
 'maddening': 8,
 'bittersweet': 9,
 'percieving': 10,
 'socionics': 11,
 'inputs': 12,
 'stir': 13,
 'adjustments': 14,
 'hopeful': 15,
 'examining': 16,
 'tmi': 17,
 'alternating': 18,
 'windows': 19,
 'lightbulb': 20,
 'prioritize': 21,
 'opinion': 22,
 'yay': 23,
 'individual': 24,
 'sela': 25,
 'fakeness': 26,
 'ww': 27,
 'depression': 28,
 'sw': 29,
 'hayley': 30,
 'professionals': 31,
 'verify': 32,
 'possession': 33,
 'volunteers': 34,
 'pauses': 35,
 'swiss': 36,
 'retard': 37,
 'alive': 38,
 'unfiltered': 39,
 'seat': 40,
 'dum': 41,
 'atheistic': 42,
 'study': 43,
 'alarmed': 44,
 'injustices': 45,
 'meant': 46,
 'combinations': 47,
 'pls': 48,
 'recognised': 49,
 'printed': 50,
 'manageable': 51,
 'prayer': 52,
 'tilt': 53,
 'curiosity': 54,
 'coelho': 55,
 'disciplined': 56,
 'speculations': 57,
 'handbsurprisedk': 58,
 'hesitantkellie': 59,
 'continuously': 60,
 '

In [22]:
x_train

array([[    0,     0,     0, ...,  6543,  8836, 14839],
       [    0,     0,     0, ..., 13904, 15974, 11956],
       [  633,  6543,  3886, ..., 13247, 18165,  3906],
       ...,
       [    0,     0,     0, ..., 11980, 18165,  8398],
       [    0,     0,     0, ..., 10304, 17516,  9483],
       [16450, 20841,   193, ..., 19459, 12343, 12478]], dtype=int32)

In [23]:
x_test

array([[    0,     0,     0, ..., 20158, 12598, 19910],
       [    0,     0,     0, ..., 19459, 15523,  3373],
       [    0,     0,     0, ...,  6181,  3707, 10849],
       ...,
       [    0,     0,     0, ...,  1256,  8938, 10836],
       [    0,     0,     0, ..., 16928, 16544,  5016],
       [    0,  3938, 17047, ..., 18600, 15775,  7532]], dtype=int32)

### Explanation and Teaching

This code cell is focused on leveraging a pre-trained BERT tokenizer to convert tokenized text data into token IDs, followed by preparing the data for training and testing a machine learning model. Here’s a breakdown:

1. **Loading the BERT Tokenizer**:
   - `tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')` initializes a pre-trained BERT tokenizer from the Hugging Face Transformers library. The `'bert-large-uncased'` model is a large version of BERT that operates on lowercase text, making it suitable for handling a variety of English language tasks. Using a pre-trained tokenizer helps in maintaining consistency with how BERT was trained.

2. **Encoding Tokens into Token IDs**:
   - `token_ids = []` initializes an empty list to store the token IDs for each post.
   - The `for` loop iterates over each list of tokens in the `dfposts['tokens']`. 
   - `temp = tokenizer.encode(...)` converts the token list into a sequence of token IDs. Key parameters include:
     - `padding='max_length'`: Ensures that all sequences are padded to a maximum length defined by `max_tokens`.
     - `truncation=True`: Truncates sequences that exceed the `max_tokens` length, maintaining input consistency.
     - `max_length=max_tokens`: Defines the maximum length for the sequences to be encoded.
     - `is_split_into_words=True`: Indicates that the input is already tokenized and split into words, which allows the tokenizer to process them appropriately.
   - The encoded token IDs for each post are appended to the `token_ids` list.

3. **Creating the Feature Array**:
   - `features = np.array(token_ids)` converts the list of token ID sequences into a NumPy array. This conversion is important for compatibility with machine learning libraries, which typically require input data to be in array format.

4. **Data Splitting**:
   - `bx_train, bx_test, by_train, by_test = train_test_split(features, target, train_size=0.9, shuffle=True)` splits the features (token IDs) and target labels into training and testing sets.
   - Similar to previous steps, 90% of the data is designated for training, and 10% is reserved for testing, with the data being shuffled to avoid any biases in the dataset.

### Teaching Points

- **Using Pre-trained Tokenizers**: Leveraging pre-trained tokenizers like BERT's allows for effective handling of tokenization, taking into account the specifics of the model's training. This is essential for ensuring that the input data aligns with the expectations of the BERT architecture.
- **Token ID Encoding**: Converting tokens to their corresponding IDs is a crucial step in preparing text for input into transformer models, as these models operate on numerical representations of text.
- **Padding and Truncation**: Consistent input length is vital for batch processing in neural networks. Padding and truncation help maintain this consistency, which is especially important for models like BERT that require fixed input sizes.
- **Feature Array Preparation**: Converting lists to NumPy arrays is often necessary for machine learning workflows, as many libraries (like TensorFlow and PyTorch) operate on array-like data structures for efficient computation.
- **Data Splitting**: Properly splitting the dataset into training and testing sets is crucial for evaluating model performance and ensuring that it generalizes well to unseen data.

Overall, this cell efficiently transforms the tokenized text into a format suitable for BERT-based models, laying the groundwork for effective model training and evaluation in the subsequent steps.

In [24]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')
token_ids = []
for token_list in dfposts['tokens']:
    temp = tokenizer.encode(text=token_list, padding='max_length', truncation=True, max_length=max_tokens, is_split_into_words=True)
    token_ids.append(temp)

features = np.array(token_ids)
bx_train, bx_test, by_train, by_test = train_test_split(features, target, train_size=0.9, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



In [25]:
dfposts.head(2)

,type,posts,tokens,I,N,F,P,token_idxs
2,INFJ,enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks,"[enfp, and, intj, moments, url, not, top, ten, plays, url, pranks]",1,1,1,0,"[5631, 1256, 8801, 417, 4892, 19440, 9970, 8922, 15819, 4892, 19772]"
3,INFJ,What has been the most life-changing experience in your life?,"[what, has, been, the, most, life, changing, etongue, sticking, out, cheeky, playful, or, blowing, a, raspberryerience, in, your, life]",1,1,1,0,"[9380, 193, 17347, 7309, 16713, 12345, 1292, 7590, 14014, 7109, 164, 18896, 18168, 1560, 4636, 5916, 6543, 15974, 12345]"


In [26]:
Y = 10  # Output dimension of LSTM cell
BS = 16 * strategy.num_replicas_in_sync  # Batch size

### Emdedding
---

### Explanation and Teaching

This code cell contains two functions that load pre-trained word embeddings: one for Word2Vec embeddings and another for GloVe embeddings. Word embeddings are crucial in natural language processing (NLP) as they provide a dense vector representation of words, capturing semantic relationships between them. Here’s a detailed breakdown of each function:

#### 1. Loading Word2Vec Embeddings

```python
def get_embedding(fname, E, binary):
    w2v = models.KeyedVectors.load_word2vec_format('/kaggle/input/{}'.format(fname), binary=binary)
    embedding_matrix = np.zeros((len(vocab) + 1, E))
    for word, i in tqdm(vocab.items(), total=len(vocab), desc='Filling Embedding Matrix'):
        if word in w2v.key_to_index:
            embedding_matrix[i] = w2v.get_vector(word)
    return Embedding(input_dim=len(vocab) + 1, output_dim=E, weights=[embedding_matrix], 
                     trainable=False, mask_zero=True, name='TokenEmbedding')
```

- **Function Definition**: `get_embedding(fname, E, binary)` defines a function that takes the filename of the Word2Vec model, the embedding dimension `E`, and a boolean `binary` indicating whether the file format is binary.

- **Loading the Model**: 
  - `models.KeyedVectors.load_word2vec_format(...)` loads the pre-trained Word2Vec embeddings from the specified file. The `binary` argument specifies if the embeddings are in binary format.

- **Initializing the Embedding Matrix**:
  - `embedding_matrix = np.zeros((len(vocab) + 1, E))` creates a matrix of zeros to store the embeddings. The size is determined by the vocabulary length plus one (to accommodate a zero index for padding).

- **Filling the Matrix**:
  - The `for` loop iterates through the vocabulary, and for each word, if it exists in the Word2Vec model, its corresponding vector is retrieved and stored in the embedding matrix.
  - `tqdm` is used to provide a progress bar for the loop, which enhances user experience during execution.

- **Returning the Embedding Layer**:
  - Finally, the function returns an `Embedding` layer initialized with the embedding matrix. This layer is set to be non-trainable (`trainable=False`), meaning the pre-trained weights will not be updated during model training.

#### 2. Loading GloVe Embeddings

```python
def load_glove_embedding(fname, E):
    embeddings_index = {}
    with open('/kaggle/input/{}'.format(fname), encoding='utf-8') as f:
        for line in tqdm(f, desc='Loading GloVe'):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print(f'Found {len(embeddings_index)} word vectors.')

    embedding_matrix = np.zeros((len(vocab) + 1, E))
    for word, i in tqdm(vocab.items(), desc='Filling Embedding Matrix'):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return Embedding(input_dim=len(vocab) + 1, output_dim=E, weights=[embedding_matrix],
                     trainable=False, mask_zero=True, name='GloVeEmbedding')
```

- **Function Definition**: `load_glove_embedding(fname, E)` defines a function that takes the filename of the GloVe model and the embedding dimension `E`.

- **Loading GloVe Vectors**:
  - An empty dictionary `embeddings_index` is initialized to store the word vectors.
  - The function reads the GloVe file line by line, splitting each line into a word and its corresponding vector. The vectors are stored in the `embeddings_index` dictionary.

- **Printing the Count**: 
  - After loading, the function prints the total number of word vectors found in the GloVe file.

- **Initializing the Embedding Matrix**: 
  - Similar to the Word2Vec function, an embedding matrix of zeros is created to store the GloVe embeddings.

- **Filling the Matrix**: 
  - The function iterates through the vocabulary and retrieves the GloVe vector for each word. If the vector exists, it is added to the embedding matrix.

- **Returning the Embedding Layer**: 
  - The function returns an `Embedding` layer initialized with the GloVe embedding matrix, which is also set to be non-trainable.

### Teaching Points

- **Understanding Word Embeddings**: Word embeddings like Word2Vec and GloVe provide a way to represent words in continuous vector space, capturing semantic relationships and context. This representation is more effective than traditional one-hot encoding.
  
- **Using Pre-trained Models**: Utilizing pre-trained embeddings saves time and resources, as they are trained on vast datasets and capture a wide range of linguistic nuances. This can significantly enhance model performance in NLP tasks.

- **Matrix Initialization**: Understanding how to create and fill embedding matrices is essential for integrating pre-trained embeddings into machine learning models.

- **Embedding Layer in Keras**: The `Embedding` layer in Keras is a powerful tool for handling categorical data, especially in NLP. Setting `trainable=False` allows for the use of rich pre-trained embeddings while preventing them from being updated during training, which can help retain the learned semantic relationships.

- **Progress Tracking**: Using libraries like `tqdm` enhances user experience by providing visual feedback during lengthy operations, which is particularly useful in data preprocessing tasks.

Overall, these functions serve as a critical step in preparing text data for training machine learning models by incorporating rich semantic information from pre-trained word embeddings.

In [27]:
# Function to load Word2Vec embeddings
def get_embedding(fname, E, binary):
    w2v = models.KeyedVectors.load_word2vec_format('/kaggle/input/{}'.format(fname), binary=binary)
    embedding_matrix = np.zeros((len(vocab) + 1, E))
    for word, i in tqdm(vocab.items(), total=len(vocab), desc='Filling Embedding Matrix'):
        if word in w2v.key_to_index:
            embedding_matrix[i] = w2v.get_vector(word)
    return Embedding(input_dim=len(vocab) + 1, output_dim=E, weights=[embedding_matrix], 
                     trainable=False, mask_zero=True, name='TokenEmbedding')

# Function to load GloVe embeddings

def load_glove_embedding(fname, E):
    embeddings_index = {}
    with open('/kaggle/input/{}'.format(fname), encoding='utf-8') as f:
        for line in tqdm(f, desc='Loading GloVe'):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print(f'Found {len(embeddings_index)} word vectors.')

    embedding_matrix = np.zeros((len(vocab) + 1, E))
    for word, i in tqdm(vocab.items(), desc='Filling Embedding Matrix'):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return Embedding(input_dim=len(vocab) + 1, output_dim=E, weights=[embedding_matrix],
                     trainable=False, mask_zero=True, name='GloVeEmbedding')


### Explanation and Teaching

This code cell is designed to create dummy inputs for building embedding layers in a neural network model using TensorFlow. Here’s a breakdown of what the code does:

#### 1. Creating Dummy Input

```python
dummy_input = tf.constant([[1, 2, 3]])
```

- **Dummy Input Definition**: The `dummy_input` tensor is created using `tf.constant`, which initializes a constant tensor in TensorFlow.
- **Shape of the Input**: The input shape here is `(1, 3)`, meaning it represents a batch size of 1 with 3 token indices. This dummy input is used to simulate input data for the embedding layers.

#### 2. Defining the Embeddings

```python
embeddings = [
    get_embedding('googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', 300, True),
    load_glove_embedding('glovetwitter27b100dtxt/glove.twitter.27B.100d.txt', 100),
]
```

- **Embeddings List**: Two embedding layers are created and stored in the `embeddings` list:
  - **Word2Vec Embedding**: The first layer is created using the `get_embedding` function for the Google News Word2Vec model with 300-dimensional vectors.
  - **GloVe Embedding**: The second layer is created using the `load_glove_embedding` function for the GloVe Twitter model with 100-dimensional vectors.

#### 3. Building the Layers

```python
for embedding in embeddings:
    _ = embedding(dummy_input)
```

- **Layer Building**: This loop iterates through each embedding layer and calls it with the `dummy_input`. This step ensures that the layers are built and the corresponding weights are initialized.
- **TensorFlow's Layer Initialization**: In TensorFlow, layers need to be called with actual data (or a dummy input) to allocate memory and define their shapes, which is crucial for model training.

#### 4. Printing the Embeddings

```python
print(embeddings)
```

- **Output**: The code prints the list of embedding layers. At this point, the embeddings contain the initialized layers with the specified pre-trained weights from the Word2Vec and GloVe models.

### Teaching Points

- **Creating Dummy Inputs**: In neural network development, creating dummy inputs is a common practice to ensure that layers are correctly built and initialized. This can help prevent shape mismatches when actual data is fed into the model.

- **Understanding Embedding Layers**: Embedding layers transform integer-encoded tokens into dense vector representations, which can capture the semantic meaning of words. This transformation is vital in NLP tasks where understanding word context is important.

- **Using Pre-trained Embeddings**: Utilizing pre-trained embeddings such as Word2Vec and GloVe allows models to leverage existing knowledge encoded in the vectors, which can lead to improved performance on NLP tasks.

- **Layer Initialization in TensorFlow**: When defining a model in TensorFlow, it's important to explicitly build layers by calling them with input data. This helps allocate resources and finalize the architecture before training begins.

- **Layer Printing**: Printing the layers can be helpful for debugging and ensuring that the model is correctly set up, providing insight into the architecture and configuration of the embedding layers.

In summary, this code cell is a crucial step in preparing the neural network model, particularly for natural language processing tasks, by setting up and initializing the embedding layers with pre-trained weights.

In [28]:
# Create a dummy input to build the layers
dummy_input = tf.constant([[1, 2, 3]])

embeddings = [
    get_embedding('googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', 300, True),
    load_glove_embedding('glovetwitter27b100dtxt/glove.twitter.27B.100d.txt', 100),
]

# Build all layers
for embedding in embeddings:
    _ = embedding(dummy_input)

# Now all layers should be built
print(embeddings)


Filling Embedding Matrix:   0%|          | 0/21611 [00:00<?, ?it/s]

Loading GloVe: 0it [00:00, ?it/s]

Found 1193514 word vectors.


Filling Embedding Matrix:   0%|          | 0/21611 [00:00<?, ?it/s]

[<Embedding name=TokenEmbedding, built=True>, <Embedding name=GloVeEmbedding, built=True>]


In [29]:
for i, embedding in enumerate(embeddings):
    print(f"Embedding {i + 1}:")
    
    # Get the weights of the embedding layer
    embedding_weights = embedding.get_weights()[0]
    
    print(f"Shape of embedding matrix: {embedding_weights.shape}")
    print(f"Type of embedding matrix: {type(embedding_weights)}")
    
    # Print first few rows of the embedding matrix
    print("First 5 rows of the embedding matrix:")
    print(embedding_weights[:2])
    
    # Print some statistics
    print(f"Mean of embedding weights: {np.mean(embedding_weights)}")
    print(f"Standard deviation of embedding weights: {np.std(embedding_weights)}")
    print(f"Min of embedding weights: {np.min(embedding_weights)}")
    print(f"Max of embedding weights: {np.max(embedding_weights)}")
    
    print("\n" + "="*50 + "\n")

# If you want to see the actual word-to-vector mapping for pre-trained embeddings:
if isinstance(embeddings[0], dict):  # Assuming the first embedding is pre-trained
    word2vec = embeddings[0]
    print("Example word vectors from Word2Vec:")
    for word in list(word2vec.keys())[:5]:  # Print first 5 words
        print(f"{word}: {word2vec[word][:10]}...")  # Print first 10 dimensions



Embedding 1:
Shape of embedding matrix: (21612, 300)
Type of embedding matrix: <class 'numpy.ndarray'>
First 5 rows of the embedding matrix:
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
 

### First Model
---

In [30]:
models = defaultdict(dict)
histories = defaultdict(dict)

### Explanation and Teaching

This code cell defines a function `get_model`, which creates and compiles a neural network model using TensorFlow's Keras API. The model is designed for tasks such as classifying text data into multiple categories based on the MBTI personality types. Below is a detailed breakdown of the code.

#### 1. Function Definition

```python
def get_model(embedding, sequence):
```

- **Function Signature**: The function `get_model` takes two parameters: `embedding`, which represents the embedding layer, and `sequence`, which likely refers to a recurrent or convolutional layer for processing sequences of text.

#### 2. Learning Rate Schedule

```python
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)
```

- **Learning Rate Decay**: The learning rate schedule is set up to decay the learning rate exponentially. The initial learning rate is `0.001`, and it decreases by a factor of `0.9` every `10000` steps. This helps in fine-tuning the model training process by allowing larger steps in the early phase and smaller steps as the training progresses.

#### 3. Model Definition Inside Strategy Scope

```python
with strategy.scope(): # Define model inside strategy scope
```

- **Distributed Training**: If using a distribution strategy (e.g., for multi-GPU training), the model is defined within the strategy scope. This ensures that the model is appropriately handled for distributed training environments.

#### 4. Sequential Model Construction

```python
model = Sequential(name="Model")
```

- **Sequential API**: A `Sequential` model is initialized, which allows layers to be added sequentially.

#### 5. Adding Layers

```python
model.add(embedding)
model.add(SpatialDropout1D(0.2, name='DroppingEmbeddings'))
model.add(sequence)
model.add(Flatten(name='FlattenLSTMOutputs'))
model.add(Dense(units=int(sqrt(max_tokens * Y)), activation='relu', name='HiddenLayer'))
model.add(Dropout(0.2, name='DroppingHiddenNodes'))
model.add(Dense(units=len(one_slice), activation='sigmoid', name='OutputLayer'))
```

- **Embedding Layer**: The first layer is the embedding layer, which transforms token indices into dense vectors.
  
- **SpatialDropout1D Layer**: A `SpatialDropout1D` layer is added to prevent overfitting by randomly dropping entire 1D feature maps. The dropout rate is set to `0.2`.
  
- **Sequence Layer**: The `sequence` layer (likely an LSTM or GRU) processes the sequences of embeddings.
  
- **Flatten Layer**: The outputs of the sequence layer are flattened to create a single vector for further processing.
  
- **Dense Layer**: A hidden dense layer is added with a number of units calculated as `sqrt(max_tokens * Y)`, using ReLU activation.
  
- **Dropout Layer**: Another dropout layer is included to further reduce the risk of overfitting.
  
- **Output Layer**: The final layer is a dense layer with a sigmoid activation function, producing output for the classification of MBTI types. The number of units corresponds to the number of MBTI indicators.

#### 6. Model Compilation

```python
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
              metrics=['binary_accuracy'], steps_per_execution=32)
```

- **Loss Function**: The model uses binary cross-entropy as the loss function, which is suitable for multi-label classification tasks.
  
- **Optimizer**: The Adam optimizer is employed with the previously defined learning rate schedule.
  
- **Metrics**: The model is evaluated using binary accuracy, which is appropriate for multi-label classification.
  
- **Steps Per Execution**: The `steps_per_execution` parameter is set to `32`, allowing efficient execution during training.

#### 7. Return Statement

```python
return model
```

- **Model Return**: The constructed and compiled model is returned for further use in training and evaluation.

### Teaching Points

- **Understanding Neural Network Architecture**: This code illustrates how to build a neural network model layer by layer, demonstrating the importance of embedding layers for text data, and the use of dropout layers to mitigate overfitting.

- **Learning Rate Scheduling**: Utilizing a learning rate schedule helps in adapting the learning rate during training, which can lead to better convergence and model performance.

- **Distributed Training**: Defining the model within a strategy scope is critical for leveraging multi-GPU or distributed training environments, making the training process more efficient.

- **Compiling Models**: The `compile` step is crucial as it configures the model for training. Selecting the appropriate loss function and optimizer is key to achieving good performance.

- **Model Customization**: This model is customizable; users can adjust the number of units in the dense layers, dropout rates, and the types of layers used depending on the complexity of the task and the dataset characteristics.

In summary, the `get_model` function encapsulates the process of constructing a comprehensive neural network model for MBTI classification based on text inputs, combining different layer types to handle the data effectively.

In [31]:
def get_model(embedding, sequence):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)
    with strategy.scope(): # Define model inside strategy scope
        model = Sequential(name="Model")
        model.add(embedding)
        model.add(SpatialDropout1D(0.2, name='DroppingEmbeddings'))
        model.add(sequence)  
        model.add(Flatten(name='FlattenLSTMOutputs'))
        model.add(Dense(units=int(sqrt(max_tokens * Y)), activation='relu', name='HiddenLayer'))
        model.add(Dropout(0.2, name='DroppingHiddenNodes'))
        model.add(Dense(units=len(one_slice), activation='sigmoid', name='OutputLayer'))
        model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                      metrics=['binary_accuracy'], steps_per_execution=32)
    return model



### Explanation and Teaching

This code snippet demonstrates how to train neural network models using different embedding techniques (Word2Vec and GloVe) along with two types of recurrent layers (GRU and LSTM). It outlines the steps to fit the models on training data and store their histories for future analysis. Here's a breakdown of the code:

#### 1. Training for GRU Layer

```python
for embedding_name, embedding in zip(['Word2Vec', 'GloVe'], embeddings):
    print('Training for {} & GRU'.format(embedding_name))
    model = get_model(embedding, GRU(units=Y, return_sequences=True, name='GRU', dropout=0.2, recurrent_dropout=0.2))
    models[embedding_name]['GRU'] = model
    cbs = [callbacks.EarlyStopping(patience=25, monitor='val_binary_accuracy')]
    history = model.fit(x_train, y_train, batch_size=BS, epochs=1, validation_data=(x_test, y_test), shuffle=True, callbacks=cbs)
    histories[embedding_name]['GRU'] = history
```

- **Looping Through Embeddings**: The loop iterates over the two embedding types, `Word2Vec` and `GloVe`, using `zip` to pair each name with its corresponding embedding.

- **Model Training for GRU**: For each embedding, a model is created using the `get_model` function, where the embedding layer is passed along with a GRU layer. The GRU layer is configured with the following parameters:
  - `units=Y`: Number of units in the GRU layer.
  - `return_sequences=True`: The GRU layer returns the full sequence of outputs, not just the last output, which is useful for stacked RNNs.
  - `dropout=0.2` and `recurrent_dropout=0.2`: Dropout rates are set to reduce overfitting, applied to both the inputs and the recurrent connections.

- **Model Storage**: The created model is stored in the `models` dictionary under the appropriate embedding name and layer type.

- **Early Stopping Callback**: An `EarlyStopping` callback is defined to monitor the validation binary accuracy and stop training if it does not improve for 25 epochs, preventing overfitting.

- **Model Fitting**: The model is trained using `model.fit` with the following parameters:
  - `x_train` and `y_train`: Training data.
  - `batch_size=BS`: Number of samples per gradient update.
  - `epochs=1`: The model will be trained for one epoch.
  - `validation_data=(x_test, y_test)`: The validation dataset for monitoring performance.
  - `shuffle=True`: Shuffles the training data before each epoch.
  - `callbacks=cbs`: The defined callbacks for early stopping.

- **Storing Training History**: The training history is saved in the `histories` dictionary for later analysis.

#### 2. Training for LSTM Layer

The same process is repeated for the LSTM layer:

```python
for embedding_name, embedding in zip(['Word2Vec', 'GloVe'], embeddings):
    print('Training for {} & LSTM'.format(embedding_name))
    model = get_model(embedding, LSTM(units=Y, return_sequences=True, name='LSTM', dropout=0.2, recurrent_dropout=0.2))
    models[embedding_name]['LSTM'] = model
    cbs = [callbacks.EarlyStopping(patience=25, monitor='val_binary_accuracy')]
    history = model.fit(x_train, y_train, batch_size=BS, epochs=1, validation_data=(x_test, y_test), shuffle=True, callbacks=cbs)
    histories[embedding_name]['LSTM'] = history
```

- **Model Training for LSTM**: The process is nearly identical to the GRU training. A model is created using the `get_model` function with the LSTM layer instead. The parameters for the LSTM layer are similar, emphasizing the flexibility of using different types of recurrent layers.

### Teaching Points

- **Understanding RNN Variants**: This code highlights the difference between GRU and LSTM layers. Both are types of recurrent neural networks, but they have different architectures that can impact performance based on the task. LSTM layers are more complex and may handle long-term dependencies better, while GRUs are simpler and faster.

- **Early Stopping**: Implementing early stopping is a best practice in training deep learning models. It helps to avoid overfitting by monitoring validation performance and halting training when no improvement is observed.

- **Model Evaluation**: By storing the training history, you can analyze the performance of each model over time. This can provide insights into which embedding and layer type combinations yield better results.

- **Flexibility with Embeddings**: The approach allows for easy experimentation with different embeddings and model architectures. Users can compare how different embeddings affect model performance.

- **Data Shuffling**: Shuffling training data before each epoch helps ensure that the model does not learn the order of the data, promoting better generalization.

### Conclusion

This code snippet effectively sets up the training process for both GRU and LSTM models using different embeddings. It emphasizes the importance of monitoring training, handling overfitting, and the flexibility to experiment with various configurations in neural network training.

In [32]:
for embedding_name, embedding in zip(['Word2Vec', 'GloVe'], embeddings):
    print('Training for {} & GRU'.format(embedding_name))
    model = get_model(embedding, GRU(units=Y, return_sequences=True, name='GRU', dropout=0.2, recurrent_dropout=0.2))
    models[embedding_name]['GRU'] = model
    cbs = [callbacks.EarlyStopping(patience=25, monitor='val_binary_accuracy')]
    history = model.fit(x_train, y_train, batch_size=BS, epochs=1, validation_data=(x_test, y_test), shuffle=True, callbacks=cbs)
    histories[embedding_name]['GRU'] = history

Training for Word2Vec & GRU


/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning:

Layer 'FlattenLSTMOutputs' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.



21601/21663 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - binary_accuracy: 0.6969 - loss: 0.5773

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning:

Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.



21663/21663 ━━━━━━━━━━━━━━━━━━━━ 418s 17ms/step - binary_accuracy: 0.6969 - loss: 0.5773 - val_binary_accuracy: 0.6982 - val_loss: 0.5613
Training for GloVe & GRU
21663/21663 ━━━━━━━━━━━━━━━━━━━━ 402s 17ms/step - binary_accuracy: 0.6927 - loss: 0.5912 - val_binary_accuracy: 0.6955 - val_loss: 0.5626


In [33]:
for embedding_name, embedding in zip(['Word2Vec', 'GloVe'], embeddings):

    
    print('Training for {} & LSTM'.format(embedding_name))
    model = get_model(embedding, LSTM(units=Y, return_sequences=True, name='LSTM', dropout=0.2, recurrent_dropout=0.2))
    models[embedding_name]['LSTM'] = model
    cbs = [callbacks.EarlyStopping(patience=25, monitor='val_binary_accuracy')]
    history = model.fit(x_train, y_train, batch_size=BS, epochs=1, validation_data=(x_test, y_test), shuffle=True, callbacks=cbs)
    histories[embedding_name]['LSTM'] = history

Training for Word2Vec & LSTM
21663/21663 ━━━━━━━━━━━━━━━━━━━━ 392s 16ms/step - binary_accuracy: 0.7006 - loss: 0.5757 - val_binary_accuracy: 0.6991 - val_loss: 0.5599
Training for GloVe & LSTM
21663/21663 ━━━━━━━━━━━━━━━━━━━━ 367s 15ms/step - binary_accuracy: 0.6987 - loss: 0.5762 - val_binary_accuracy: 0.7035 - val_loss: 0.5560


---

In [34]:
models

defaultdict(dict,
            {'Word2Vec': {'GRU': <Sequential name=Model, built=True>,
              'LSTM': <Sequential name=Model, built=True>},
             'GloVe': {'GRU': <Sequential name=Model, built=True>,
              'LSTM': <Sequential name=Model, built=True>}})

In [35]:
histories

defaultdict(dict,
            {'Word2Vec': {'GRU': <keras.src.callbacks.history.History at 0x7c6ec4de24a0>,
              'LSTM': <keras.src.callbacks.history.History at 0x7c6e39eee0e0>},
             'GloVe': {'GRU': <keras.src.callbacks.history.History at 0x7c6d5e90c190>,
              'LSTM': <keras.src.callbacks.history.History at 0x7c6e3f5810f0>}})

In [36]:
# Save each model in HDF5 format
for embedding_name, model_dict in models.items():
    for model_name, model in model_dict.items():
        model.save(f'{embedding_name}_{model_name}.h5')  # Save as .h5
